In [31]:
%autoreload 2 
import numpy as np 
from pianoq.misc.mplt import mplot, mimshow
from pianoq.simulations.abstract_quantum_scaling.slm3_simulation import QWFSSimulation
# s = QWFSSimulation(N=32, T_method='gaus_iid')
N_modes = 32
s = QWFSSimulation(N=N_modes, T_method='unitary')


algos = ['slsqp', "L-BFGS-B", 'simulated_annealing', 'genetic_algorithm', 'PSO']
configs = ['SLM1', 'SLM2', 'SLM3']
T_methods = ['unitary', 'gaus_iid']

N_tries = 1
N_algos = len(algos)  # 
N_configs = len(configs)
N_T_methods = len(T_methods)

results = np.zeros((N_T_methods, N_configs, N_tries, N_algos))
best_phases = np.zeros((N_T_methods, N_configs, N_tries, N_algos, s.N))
Ts = []

for try_no in range(N_tries):
    print(f'{try_no=}')
    for T_method_no, T_method in enumerate(T_methods):
        s.T_method = T_method
        s.reset_T()
        Ts.append(s.T)
        for config_no, config in enumerate(configs):
            for algo_no, algo in enumerate(algos):
                s.config = config
                s.slm_phases = np.exp(1j*np.zeros(s.N, dtype=np.complex128))
                I, res = s.optimize(algo=algo)
                v_out = s.propagate()
                I_tot = (np.abs(v_out)**2).sum() 
                I_good = np.abs(v_out[s.N//2])**2
                # print(rf'{method=}, {I_tot=:.4f}, {I_good=:.4f}, {s.f_calls=}')
                results[T_method_no, config_no, try_no, algo_no] = I_good
                best_phases[T_method_no, config_no, try_no, algo_no] = np.angle(s.slm_phases)

try_no=0
Done initializing random average cost
Done initializing random average cost
Done initializing random average cost
Done initializing random average cost
Done initializing random average cost
Done initializing random average cost


In [33]:
# results = np.zeros((N_configs, N_tries, N_algos))
# results = np.load(r'C:\temp\qwfs_results.npy')
import matplotlib.pyplot as plt 
fig, axes = plt.subplots(N_configs, N_T_methods)
for config_no, config in enumerate(configs):
    for T_method_no, T_method in enumerate(T_methods):
        # upper_lim = 1 if T_method == 'unitary' else 2 
        # imm = axes[config_no, T_method_no].imshow(results[T_method_no, config_no], clim=(0, upper_lim))
        imm = axes[config_no, T_method_no].imshow(results[T_method_no, config_no])
        axes[config_no, T_method_no].set_title(rf'{config}, {T_method}')
        fig.colorbar(imm, ax=axes[config_no, T_method_no])
fig.show()

In [26]:
np.save(r'C:\temp\qwfs_results.npy', results)

In [ ]:
for method, r in zip(algos, results.T):
    print(f'{method:<20} {r.mean():.3f} +- {r.std():.3f}')

In [153]:
mplot(np.abs(v_out)**2)

(<Figure size 640x480 with 1 Axes>, <AxesSubplot: >)

In [29]:
display(np.pi/4)
display((np.pi/4)**2)

0.7853981633974483

0.6168502750680849